<a href="https://colab.research.google.com/github/BUN-Nakai/MachineLearningAlgorithm/blob/main/multiple_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 重回帰分析



### 線形回帰の前提
1. 線形性
2. 内生性がない
3. 正規性と同分散性
4. 自己相関がない
5. 多重共線性がない


### ダミー変数トラップ
多重共線性が発生してしまう → 他の変数で説明できてしまう<br>
例 : a+b+c=1 → a=1-b-c

他の変数をほかの変数で説明できる場合は、片方を削除する必要がある

### P値
ある事象が起こる確率

仮説：コインが歪ではない（表：0.5 裏：0.5）

裏が出続ける確率<br>
1回目: 0.5<br>
2回目: 0.25<br>
3回目: 0.125<br>
4回目: 0.06<br>
5回目: 0.03<br>
6回目: 0.01<br>
→この仮説は正しくない可能性がある<br>

確率のことがP値<br>
一般的にP値が0.05以下の場合は、この仮説を棄却する<br>

<b>線形回帰では、係数の値が0である事を仮説とする</b><br>
→その値が0.05以下ならそのその仮説を棄却する<br>
→つまり、その係数は0ではない<br>
<b>→回帰を行う為の重要な係数である</b><br>

### 変数の抽出の必要性
1. モデルをシンプルにするため
2. 定性的に評価を行えるようにするため

### 抽出の方法
**1. All-in**<br>
全ての変数を入れてモデルを作成<br>
事前の情報がない場合<br>

**2. Backward Elimination(変数減少法**）<br>
Step1. モデルをそとのままにするため、有意水準(一般的に0.05)を決める<br>
Step2. すべての独立変数を使ってモデルを作成する<br>
Step3. p値が最も大きい独立変数を見つけ、そのp値が有意水準よりも大きければ、Step4へ<br>
小さければモデル完成<br>
Step4. その独立変数を取り除く<br>
Step5. 取り除いたうえで改めてモデルを作成する

**3. Forward Selection(変数増加法)**<br>
Step1. モデルにその変数を入れる為の閾値として有意水準を決める<br>
Step2. すべての独立変数において、最もp値が低い変数を求める<br>
Step3. その変数を追加したうえで改めてどれぞれの独立変数においてモデルを作成する<br>
Step4. その中でp値が最も低い独立変数を見つけ、その値が有意水準よりも小さければモデルにその変数を入れ、それがなければモデルを完成させる<br>

**4. Bidirectional elimination(変数増減法)**<br>
Step1. モデルにその変数を入れる・取り除くための閾値としての有意水準を決める
Step2. Forward Selectionを行う<br>
Step3. Backward Eliminationを行う<br>
Step4. 取り除く、もしくは追加する変数がなくなった時点でモデルを完成させる<br>

**5. All Possible Models**<br>
Step1. モデルの当てはまりの良さを評価する為の基準を選択する（赤池情報量基準など）<br>
Step2. すべての取りえる変数において回帰モデルを作成する<br>
Step3. 最もスコアが高いモデルを採用する<br>

※ *変数が多い場合、演算負荷が高くなる*


## ライブラリのインポート

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## データセットのインポート

In [3]:
dataset = pd.read_csv('50_Startups.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [4]:
X

array([[165349.2, 136897.8, 471784.1, 'New York'],
       [162597.7, 151377.59, 443898.53, 'California'],
       [153441.51, 101145.55, 407934.54, 'Florida'],
       [144372.41, 118671.85, 383199.62, 'New York'],
       [142107.34, 91391.77, 366168.42, 'Florida'],
       [131876.9, 99814.71, 362861.36, 'New York'],
       [134615.46, 147198.87, 127716.82, 'California'],
       [130298.13, 145530.06, 323876.68, 'Florida'],
       [120542.52, 148718.95, 311613.29, 'New York'],
       [123334.88, 108679.17, 304981.62, 'California'],
       [101913.08, 110594.11, 229160.95, 'Florida'],
       [100671.96, 91790.61, 249744.55, 'California'],
       [93863.75, 127320.38, 249839.44, 'Florida'],
       [91992.39, 135495.07, 252664.93, 'California'],
       [119943.24, 156547.42, 256512.92, 'Florida'],
       [114523.61, 122616.84, 261776.23, 'New York'],
       [78013.11, 121597.55, 264346.06, 'California'],
       [94657.16, 145077.58, 282574.31, 'New York'],
       [91749.16, 114175.79, 29491

In [6]:
y

array([192261.83, 191792.06, 191050.39, 182901.99, 166187.94, 156991.12,
       156122.51, 155752.6 , 152211.77, 149759.96, 146121.95, 144259.4 ,
       141585.52, 134307.35, 132602.65, 129917.04, 126992.93, 125370.37,
       124266.9 , 122776.86, 118474.03, 111313.02, 110352.25, 108733.99,
       108552.04, 107404.34, 105733.54, 105008.31, 103282.38, 101004.64,
        99937.59,  97483.56,  97427.84,  96778.92,  96712.8 ,  96479.51,
        90708.19,  89949.14,  81229.06,  81005.76,  78239.91,  77798.83,
        71498.49,  69758.98,  65200.33,  64926.08,  49490.75,  42559.73,
        35673.41,  14681.4 ])

## カテゴリ変数のエンコーディング

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[("encoder",  OneHotEncoder(), [3])], remainder="passthrough")
X = np.array(ct.fit_transform(X))

In [8]:
# 変換した値が一番最初の列に来る
X

array([[0.0, 0.0, 1.0, 165349.2, 136897.8, 471784.1],
       [1.0, 0.0, 0.0, 162597.7, 151377.59, 443898.53],
       [0.0, 1.0, 0.0, 153441.51, 101145.55, 407934.54],
       [0.0, 0.0, 1.0, 144372.41, 118671.85, 383199.62],
       [0.0, 1.0, 0.0, 142107.34, 91391.77, 366168.42],
       [0.0, 0.0, 1.0, 131876.9, 99814.71, 362861.36],
       [1.0, 0.0, 0.0, 134615.46, 147198.87, 127716.82],
       [0.0, 1.0, 0.0, 130298.13, 145530.06, 323876.68],
       [0.0, 0.0, 1.0, 120542.52, 148718.95, 311613.29],
       [1.0, 0.0, 0.0, 123334.88, 108679.17, 304981.62],
       [0.0, 1.0, 0.0, 101913.08, 110594.11, 229160.95],
       [1.0, 0.0, 0.0, 100671.96, 91790.61, 249744.55],
       [0.0, 1.0, 0.0, 93863.75, 127320.38, 249839.44],
       [1.0, 0.0, 0.0, 91992.39, 135495.07, 252664.93],
       [0.0, 1.0, 0.0, 119943.24, 156547.42, 256512.92],
       [0.0, 0.0, 1.0, 114523.61, 122616.84, 261776.23],
       [1.0, 0.0, 0.0, 78013.11, 121597.55, 264346.06],
       [0.0, 0.0, 1.0, 94657.16, 145077.58

今回のデータは欠損値の対応は必要か？<br>
→　研究開発費、広告費がかからない企業である可能性もある

## 訓練用データセットとテスト用データセットへの分割

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [10]:
X_train

array([[0.0, 1.0, 0.0, 55493.95, 103057.49, 214634.81],
       [0.0, 0.0, 1.0, 46014.02, 85047.44, 205517.64],
       [0.0, 1.0, 0.0, 75328.87, 144135.98, 134050.07],
       [1.0, 0.0, 0.0, 46426.07, 157693.92, 210797.67],
       [0.0, 1.0, 0.0, 91749.16, 114175.79, 294919.57],
       [0.0, 1.0, 0.0, 130298.13, 145530.06, 323876.68],
       [0.0, 1.0, 0.0, 119943.24, 156547.42, 256512.92],
       [0.0, 0.0, 1.0, 1000.23, 124153.04, 1903.93],
       [0.0, 0.0, 1.0, 542.05, 51743.15, 0.0],
       [0.0, 0.0, 1.0, 65605.48, 153032.06, 107138.38],
       [0.0, 0.0, 1.0, 114523.61, 122616.84, 261776.23],
       [0.0, 1.0, 0.0, 61994.48, 115641.28, 91131.24],
       [1.0, 0.0, 0.0, 63408.86, 129219.61, 46085.25],
       [1.0, 0.0, 0.0, 78013.11, 121597.55, 264346.06],
       [1.0, 0.0, 0.0, 23640.93, 96189.63, 148001.11],
       [1.0, 0.0, 0.0, 76253.86, 113867.3, 298664.47],
       [0.0, 0.0, 1.0, 15505.73, 127382.3, 35534.17],
       [0.0, 0.0, 1.0, 120542.52, 148718.95, 311613.29],
       

## 訓練用データセットを使ったモデルの学習

In [11]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

lr.fit(X_train, y_train)

LinearRegression()

In [ ]:
# ダミー変数トラップの対応（Stateの対応）
# 本来は対応しないといけないが、sklearn内で処理してくれるので、
# 特にコードで明示的に削除する必要はない

In [ ]:
# すべての独立変数を使用しているが、P値が大きいものを削除し
# モデルをシンプルにする方がよいのでは？
# 上記は正しいが、sklearn内でそれも処理してくれるので
# コードで明示的に行う必要がない

## テストデータセットに対する予測

In [12]:
y_test

array([103282.38, 144259.4 , 146121.95,  77798.83, 191050.39, 105008.31,
        81229.06,  97483.56, 110352.25, 166187.94])

In [13]:
y_pred = lr.predict(X_test)

In [14]:
y_pred

array([103015.20159795, 132582.27760816, 132447.73845175,  71976.09851258,
       178537.48221057, 116161.24230167,  67851.69209676,  98791.73374687,
       113969.43533014, 167921.06569552])

In [17]:
# 性能評価用
# from sklearn import metrics
# metrics.accuracy_score(y_test, y_pred)
# 線形回帰には上記はサポートされていない？
# No metrics support "multiclass-multioutput" format
# ValueError: continuous is not supported

In [19]:
# 小数点の桁を合わせる(有効桁数2)
np.set_printoptions(precision=2)
# 行列を結合(行ベクトルと列ベクトルに変換して結合)
np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1)

array([[103015.2 , 103282.38],
       [132582.28, 144259.4 ],
       [132447.74, 146121.95],
       [ 71976.1 ,  77798.83],
       [178537.48, 191050.39],
       [116161.24, 105008.31],
       [ 67851.69,  81229.06],
       [ 98791.73,  97483.56],
       [113969.44, 110352.25],
       [167921.07, 166187.94]])

Pythonで実装をする中では、変数減少法の実装方法について理解している必要はありません。

なぜなら、scikitlearnが内部での演算を通じて重要な変数だけを抽出してくれるからです。

**変数減少法について実装**

    # ライブラリのインポート
     
    import numpy as np
    import matplotlib.pyplot as plt
    import pandas as pd
     
     
     
    # データセットのインポート
    dataset = pd.read_csv('50_Startups.csv')
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, -1].values
    print(X)
     
     
     
    # カテゴリ変数のエンコーディング
    from sklearn.compose import ColumnTransformer
    from sklearn.preprocessing import OneHotEncoder
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [3])], remainder='passthrough')
    X = np.array(ct.fit_transform(X))
    print(X)
     
     
     
    # ダミー変数トラップを避けるためのコード
    X = X[:, 1:]
      
    # 訓練用とテスト用へのデータセットの分割
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
     

    # 訓練用データセットを使った重回帰モデルの訓練
    from sklearn.linear_model import LinearRegression
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
     
    # テスト用データを使った結果の予測
    y_pred = regressor.predict(X_test)
    np.set_printoptions(precision=2)
    print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
     
    # 変数減少法を使った変数の抽出
    import statsmodels.api as sm
    X = np.append(arr = np.ones((50, 1)).astype(int), values = X, axis = 1)
    X_opt = X[:, [0, 1, 2, 3, 4, 5]]
    X_opt = X_opt.astype(np.float64)
    regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
    regressor_OLS.summary()
     
    X_opt = X[:, [0, 1, 3, 4, 5]]
    X_opt = X_opt.astype(np.float64)
     
    regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
    regressor_OLS.summary()
     
    X_opt = X[:, [0, 3, 4, 5] 
    X_opt = X_opt.astype(np.float64)
     
    regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
    regressor_OLS.summary()X_opt = X[:, [0, 3, 5]]
    X_opt = X_opt.astype(np.float64)
     
    regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
    regressor_OLS.summary()
     
    X_opt = X[:, [0, 3]]
    X_opt = X_opt.astype(np.float64)
    regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
    regressor_OLS.summary()